In [ ]:
#default_exp tabular.core

In [ ]:
#export
from local.torch_basics import *
from local.test import *
from local.core import *
from local.data.all import *
from local.notebook.showdoc import show_doc

In [ ]:
#export
pd.set_option('mode.chained_assignment','raise')

# Tabular core

> Basic function to preprocess tabular data before assembling it in a `DataBunch`.

## Tabular -

In [ ]:
#export
class _TabIloc:
    "Get/set rows by iloc and cols by name"
    def __init__(self,to): self.to = to
    def __getitem__(self, idxs):
        if isinstance(idxs,tuple):
            rows,cols = idxs
            c = self.to.items.columns
            cols = c.isin(cols) if is_listy(cols) else c.get_loc(cols)
        else: rows,cols = idxs,slice(None)
        return self.to.new(self.to.items.iloc[rows, cols])

In [ ]:
#export
class Tabular(CollBase):
    "A `DataFrame` wrapper that knows which cols are cont/cat/y, and returns rows in `__getitem__`"
    def __init__(self, df, procs=None, cat_names=None, cont_names=None, y_names=None, is_y_cat=True,
                 splits=None, split=None, setup=True, inplace=True):
        if not inplace: df = df.copy()
        if splits is not None: df = df.iloc[sum(splits, [])].reset_index(drop=True)
        super().__init__(df)
        if split is None and splits is not None: split = len(splits[0])
        store_attr(self, 'y_names,is_y_cat,split')
        self.cat_names,self.cont_names,self.procs = L(cat_names),L(cont_names),Pipeline(procs)
        self.cat_y  = None if not is_y_cat else y_names
        self.cont_y = None if     is_y_cat else y_names
        if setup: self.procs.setup(self)

    def new(self, df):
        return self.__class__(df, self.procs, self.cat_names, self.cont_names, self.y_names, is_y_cat=self.is_y_cat,
                              split=self.split, setup=False)

    def __getattr__(self,k): return delegate_attr(self, k, 'items')
    def show(self, max_n=10, **kwargs): display_df(self.all_cols[:max_n])
    def process(self): self.procs(self)

    @property
    def iloc(self): return _TabIloc(self)
    @property
    def targ(self): return self.items[self.y_names]
    @property
    def all_cont_names(self): return self.cont_names + self.cont_y
    @property
    def all_cat_names (self): return self.cat_names  + self.cat_y
    @property
    def all_col_names (self): return self.all_cont_names + self.all_cat_names

In [ ]:
#export
class TabularPandas(Tabular):
    def transform(self, cols, f): self[cols] = self[cols].transform(f)

In [ ]:
#export
def _add_prop(cls, nm):
    prop = property(lambda o: o[list(getattr(o,nm+'_names'))])
    setattr(cls, nm+'s', prop)
    def _f(o,v): o[getattr(o,nm+'_names')] = v
    setattr(cls, nm+'s', prop.setter(_f))

_add_prop(Tabular, 'cat')
_add_prop(Tabular, 'all_cat')
_add_prop(Tabular, 'cont')
_add_prop(Tabular, 'all_cont')
_add_prop(Tabular, 'all_col')

In [ ]:
df = pd.DataFrame({'a':[0,1,2,0,2], 'b':[0,0,0,0,1]})
to = TabularPandas(df, cat_names='a')
t = pickle.loads(pickle.dumps(to))
test_eq(t.items,to.items)
test_eq(to.all_cols,to[['a']])
to.show() # only shows 'a' since that's the only col in `TabularPandas`

,a
0,0
1,1
2,2
3,0
4,2


In [ ]:
#export
class TabularProc(InplaceTransform):
    "Base class to write a non-lazy tabular processor for dataframes"
    def setup(self, items=None):
        super().setup(items)
        # Procs are called as soon as data is available
        return self(items)

In [ ]:
#export
class Categorify(TabularProc):
    "Transform the categorical variables to that type."
    order = 1
    def setups(self, to):
        self.classes = {n:CategoryMap(to.iloc[:to.split,n], add_na=True) for n in to.all_cat_names}

    def _apply_cats (self, c): return c.cat.codes+1 if is_categorical_dtype(c) else c.map(self[c.name].o2i)
    def _decode_cats(self, c): return c.map(dict(enumerate(self[c.name].items)))
    def encodes(self, to): to.transform(to.all_cat_names, self._apply_cats)
    def decodes(self, to): to.transform(to.all_cat_names, self._decode_cats)
    def __getitem__(self,k): return self.classes[k]

In [ ]:
show_doc(Categorify, title_level=3)

<h3 id="Categorify" class="doc_header"><code>class</code> <code>Categorify</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>Categorify</code>(**`enc`**=*`None`*, **`dec`**=*`None`*, **`filt`**=*`None`*, **`as_item`**=*`False`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Transform the categorical variables to that type.

In [ ]:
df = pd.DataFrame({'a':[0,1,2,0,2]})
to = TabularPandas(df, Categorify, 'a')
cat = to.procs.categorify
test_eq(cat['a'], ['#na#',0,1,2])
test_eq(df['a'], [1,2,3,1,3])

In [ ]:
df1 = pd.DataFrame({'a':[1,0,3,-1,2]})
to1 = to.new(df1)
to1.process()
#Values that weren't in the training df are sent to 0 (na)
test_eq(df1['a'], [2,1,0,0,3])
to2 = cat.decode(to1)
test_eq(to2.a, [1,0,'#na#','#na#',2])

In [ ]:
#test with splits
df = pd.DataFrame({'a':[0,1,2,3,2]})
to = TabularPandas(df, Categorify, 'a', split=3)
cat = to.procs[0]
test_eq(cat['a'], ['#na#',0,1,2])
test_eq(df['a'], [1,2,3,0,3])

In [ ]:
df = pd.DataFrame({'a':pd.Categorical(['M','H','L','M'], categories=['H','M','L'], ordered=True)})
to = TabularPandas(df, Categorify, 'a')
cat = to.procs[0]
test_eq(cat['a'], ['#na#','H','M','L'])
test_eq(df.a, [2,1,3,2])
to2 = cat.decode(to)
test_eq(to2.a, ['M','H','L','M'])

In [ ]:
#export
class Normalize(TabularProc):
    "Normalize the continuous variables."
    order = 2
    def setups(self, to):
        df = to.iloc[:to.split, to.cont_names]
        self.means,self.stds = df.mean(),df.std(ddof=0)+1e-7

    def encodes(self, to): to.conts = (to.conts-self.means) / self.stds
    def decodes(self, to): to.conts = (to.conts*self.stds ) + self.means

In [ ]:
show_doc(Normalize, title_level=3)

<h3 id="Normalize" class="doc_header"><code>class</code> <code>Normalize</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>Normalize</code>(**`enc`**=*`None`*, **`dec`**=*`None`*, **`filt`**=*`None`*, **`as_item`**=*`False`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Normalize the continuous variables.

In [ ]:
norm = Normalize()
df = pd.DataFrame({'a':[0,1,2,3,4]})
to = TabularPandas(df, norm, cont_names='a')
x = np.array([0,1,2,3,4])
m,s = x.mean(),x.std()
test_eq(norm.means['a'], m)
test_close(norm.stds['a'], s)
test_close(df['a'].values, (x-m)/s)

In [ ]:
df1 = pd.DataFrame({'a':[5,6,7]})
to1 = to.new(df1)
to1.process()
test_close(df1['a'].values, (np.array([5,6,7])-m)/s)
to2 = norm.decode(to1)
test_close(to2.a.values, [5,6,7])

In [ ]:
norm = Normalize()
df = pd.DataFrame({'a':[0,1,2,3,4]})
to = TabularPandas(df, norm, cont_names='a', split=3)
x = np.array([0,1,2])
m,s = x.mean(),x.std()
test_eq(norm.means['a'], m)
test_close(norm.stds['a'], s)
test_close(df['a'].values, (np.array([0,1,2,3,4])-m)/s)

In [ ]:
#export
class FillStrategy:
    "Namespace containing the various filling strategies."
    def median  (c,fill): return c.median()
    def constant(c,fill): return fill
    def mode    (c,fill): return c.dropna().value_counts().idxmax()

In [ ]:
#export
class FillMissing(TabularProc):
    "Fill the missing values in continuous columns."
    def __init__(self, fill_strategy=FillStrategy.median, add_col=True, fill_vals=None):
        if fill_vals is None: fill_vals = defaultdict(int)
        store_attr(self, 'fill_strategy,add_col,fill_vals')

    def setups(self, to):
        df = to.iloc[:to.split, to.cont_names].items
        self.na_dict = {n:self.fill_strategy(df[n], self.fill_vals[n])
                        for n in pd.isnull(to.conts).any().keys()}

    def encodes(self, to):
        missing = pd.isnull(to.conts)
        for n in missing.any().keys():
            assert n in self.na_dict, f"nan values in `{n}` but not in setup training set"
            to[n].fillna(self.na_dict[n], inplace=True)
            if self.add_col:
                to.loc[:,n+'_na'] = missing[n]
                if n+'_na' not in to.cat_names: to.cat_names.append(n+'_na')

In [ ]:
show_doc(FillMissing, title_level=3)

<h3 id="FillMissing" class="doc_header"><code>class</code> <code>FillMissing</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>FillMissing</code>(**`fill_strategy`**=*`'median'`*, **`add_col`**=*`True`*, **`fill_vals`**=*`None`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Fill the missing values in continuous columns.

In [ ]:
fill1,fill2,fill3 = (FillMissing(fill_strategy=s) 
                     for s in [FillStrategy.median, FillStrategy.constant, FillStrategy.mode])
df = pd.DataFrame({'a':[0,1,np.nan,1,2,3,4]})
df1 = df.copy(); df2 = df.copy()
to,to1,to2 = TabularPandas(df, fill1, cont_names='a'),TabularPandas(df1, fill2, cont_names='a'),TabularPandas(df2, fill3, cont_names='a')
test_eq(fill1.na_dict, {'a': 1.5})
test_eq(fill2.na_dict, {'a': 0})
test_eq(fill3.na_dict, {'a': 1.0})

for t in [to, to1, to2]: test_eq(t.cat_names, ['a_na'])

for to_,v in zip([to, to1, to2], [1.5, 0., 1.]):
    test_eq(to_.a.values, np.array([0, 1, v, 1, 2, 3, 4]))
    test_eq(to_.a_na.values, np.array([0, 0, 1, 0, 0, 0, 0]))

In [ ]:
dfa = pd.DataFrame({'a':[np.nan,0,np.nan]})
dfa1 = dfa.copy(); dfa2 = dfa.copy()
to,to1,to2 = to.new(dfa),to1.new(dfa1),to2.new(dfa2)
to.process(); to1.process(); to2.process()
for to_,v in zip([to, to1, to2], [1.5, 0., 1.]):
    test_eq(to_.a.values, np.array([v, 0, v]))
    test_eq(to_.a_na.values, np.array([1, 0, 1]))

## TabularPandas Pipelines -

In [ ]:
procs = [Normalize, Categorify, FillMissing, noop]
df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4]})
to = TabularPandas(df, procs, 'a', 'b')

#Test setup and apply on df_trn
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to.a, [1,2,3,2,2,3,1])
test_eq(to.b_na, [1,1,2,1,1,1,1])
x = np.array([0,1,1.5,1,2,3,4])
m,s = x.mean(),x.std()
test_close(to.b.values, (x-m)/s)
test_eq(to.procs.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True]})

In [ ]:
#Test apply on y_names
df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4], 'c': ['b','a','b','a','a','b','a']})
to = TabularPandas(df, procs, 'a', 'b', y_names='c')
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to.a, [1,2,3,2,2,3,1])
test_eq(to.b_na, [1,1,2,1,1,1,1])
test_eq(to.c, [2,1,2,1,1,2,1])
x = np.array([0,1,1.5,1,2,3,4])
m,s = x.mean(),x.std()
test_close(to.b.values, (x-m)/s)
test_eq(to.procs.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True], 'c': ['#na#','a','b']})

In [ ]:
test_eq({'a':0, 'b':1}, ['a', 'b'])

AssertionError: ==:
{'a': 0, 'b': 1}
['a', 'b']

In [ ]:
x = L(['a', 'b'])

In [ ]:
hasattr(x, '__array_eq__')

False

In [ ]:
test_eq(L(['a', 'b']), {'a':0, 'b':1})

In [ ]:
to.procs.classes

(#3) [a,b_na,c]

In [ ]:
df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4], 'c': ['b','a','b','a','a','b','a']})
to = TabularPandas(df, procs, 'a', 'b', y_names='c', inplace=False)
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to.a, [1,2,3,2,2,3,1])
test_eq(df.a.dtype,int)
test_eq(to.b_na, [1,1,2,1,1,1,1])
test_eq(to.c, [2,1,2,1,1,2,1])

In [ ]:
df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,np.nan,1,1,2,3,4], 'c': ['b','a','b','a','a','b','a']})
to = TabularPandas(df, procs, cat_names='a', cont_names='b', y_names='c', inplace=True, splits=[[0,1,4,6], [2,3,5]])
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to.a, [1,2,2,1,0,2,0])
test_eq(df.a.dtype,int)
test_eq(to.b_na, [1,2,1,1,1,1,1])
test_eq(to.c, [2,1,1,1,2,1,2])

Pass the same `splits` as you will use for splitting the data, so that the setup is only done on the training set. `cat_names` are the names of the categorical variables, `cont_names` the continous ones, `y_names` are the names of the dependent variables that are categories. If `inplace=True`, processing is applied inplace, otherwis it creates a copy of `df`.

In [ ]:
#export
class ReadTabBatch(ItemTransform):
    def __init__(self, to): self.to = to
    def encodes(self, to): return (tensor(to.cats).long(),tensor(to.conts).float()), tensor(to.targ).long()

    def decodes(self, o):
        (cats,conts),targs = to_np(o)
        vals = np.concatenate([cats,conts,targs[:,None]], axis=1)
        df = pd.DataFrame(vals, columns=self.to.cat_names+self.to.cont_names+self.to.y_names)
        to = self.to.new(df)
        to = self.to.procs.decode(to)
        return to

In [ ]:
#export
@delegates()
class TabDataLoader(TfmdDL):
    do_item = noops
    def __init__(self, dataset, bs=16, shuffle=False, after_batch=None, num_workers=0, **kwargs):
        after_batch = L(after_batch)+ReadTabBatch(dataset.items)
        super().__init__(dataset, bs=bs, shuffle=shuffle, after_batch=after_batch, num_workers=num_workers, **kwargs)

    def create_batch(self, b): return self.dataset.items.iloc[b]

## Integration example

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
df_trn,df_tst = df.iloc[:10000].copy(),df.iloc[10000:].copy()
df_trn.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]
splits = RandomSplitter()(range_of(df_trn))

In [ ]:
%time to = TabularPandas(df_trn, procs, cat_names, cont_names, y_names="salary", splits=splits)

CPU times: user 425 ms, sys: 134 µs, total: 425 ms
Wall time: 424 ms


In [ ]:
filts = [list(range(0, to.split)), list(range(to.split, 10000))]
dsrc = DataSource(to, filts=filts, tfms=[None])
dl = TabDataLoader(dsrc.valid, bs=64, num_workers=0)
dl.show_batch()

,age,fnlwgt,education-num,workclass,education,marital-status,occupation,relationship,race,age_na,fnlwgt_na,education-num_na,salary
0,26.000000,244371.999479,10.0,Private,HS-grad,Married-civ-spouse,#na#,Wife,White,False,False,True,>=50k
1,54.000001,204397.000009,10.0,Private,Some-college,Married-civ-spouse,Farming-fishing,Husband,White,False,False,False,<50k
2,27.000000,233724.000080,9.0,Self-emp-inc,HS-grad,Separated,Adm-clerical,Not-in-family,White,False,False,False,<50k
3,54.000001,21697.999540,6.0,Private,10th,Divorced,Other-service,Unmarried,White,False,False,False,<50k
4,20.000000,224640.001014,12.0,Private,Assoc-acdm,Never-married,Exec-managerial,Own-child,White,False,False,False,<50k
5,22.999999,235071.000038,10.0,Private,Some-college,Never-married,#na#,Own-child,White,False,False,True,<50k
6,83.999999,188328.000183,9.0,Private,HS-grad,Widowed,Prof-specialty,Not-in-family,White,False,False,False,<50k
7,35.000000,297574.001261,12.0,Private,Assoc-acdm,Married-civ-spouse,Exec-managerial,Husband,White,False,False,False,>=50k
8,30.000000,146160.998577,10.0,Self-emp-not-inc,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,False,False,True,>=50k
9,76.000001,93124.998694,9.0,Private,HS-grad,Married-civ-spouse,Sales,Husband,White,False,False,False,<50k


In [ ]:
to_tst = to.new(df_tst)
to_tst.process()
to_tst.all_cols.head()

,age,fnlwgt,education-num,workclass,education,marital-status,occupation,relationship,race,age_na,fnlwgt_na,education-num_na,salary
10000,0.468511,1.358824,1.174389,5,10,3,2,1,2,1,1,1,1
10001,-0.927370,1.271181,-0.428872,5,12,3,15,1,4,1,1,1,1
10002,1.056250,0.160645,-1.230503,5,2,1,9,2,5,1,1,1,1
10003,0.541978,-0.275546,-0.428872,5,12,7,2,5,5,1,1,1,1
10004,0.762380,1.462089,0.372758,6,9,3,5,1,5,1,1,1,2


## Not being used now - for multi-modal

In [ ]:
class TensorTabular(TupleBase):
    def get_ctxs(self, max_n=10, **kwargs):
        n_samples = min(self[0].shape[0], max_n)
        df = pd.DataFrame(index = range(n_samples))
        return [df.iloc[i] for i in range(n_samples)]

    def display(self, ctxs): display_df(pd.DataFrame(ctxs))

class TabularLine(pd.Series):
    "A line of a dataframe that knows how to show itself"
    def show(self, ctx=None, **kwargs): return self if ctx is None else ctx.append(self)

class ReadTabLine(ItemTransform):
    def __init__(self, proc): self.proc = proc

    def encodes(self, row):
        cats,conts = (o.mapped(row.__getitem__) for o in (self.proc.cat_names,self.proc.cont_names))
        return TensorTabular(tensor(cats).long(),tensor(conts).float())

    def decodes(self, o):
        to = TabularPandas(o, self.proc.cat_names, self.proc.cont_names, self.proc.y_names)
        to = self.proc.decode(to)
        return TabularLine(pd.Series({c: v for v,c in zip(to.items[0]+to.items[1], self.proc.cat_names+self.proc.cont_names)}))

class ReadTabTarget(ItemTransform):
    def __init__(self, proc): self.proc = proc
    def encodes(self, row): return row[self.proc.y_names].astype(np.int64)
    def decodes(self, o): return Category(self.proc.classes[self.proc.y_names][o])

In [ ]:
# tds = TfmdDS(to.items, tfms=[[ReadTabLine(proc)], ReadTabTarget(proc)])
# enc = tds[1]
# test_eq(enc[0][0], tensor([2,1]))
# test_close(enc[0][1], tensor([-0.628828]))
# test_eq(enc[1], 1)

# dec = tds.decode(enc)
# assert isinstance(dec[0], TabularLine)
# test_close(dec[0], pd.Series({'a': 1, 'b_na': False, 'b': 1}))
# test_eq(dec[1], 'a')

# test_stdout(lambda: print(tds.show_at(1)), """a               1
# b_na        False
# b               1
# category        a
# dtype: object""")

## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 01a_torch_core.ipynb.
Converted 01b_script.ipynb.
Converted 01c_dataloader.ipynb.
Converted 02_data_transforms.ipynb.
Converted 03_data_pipeline.ipynb.
Converted 05_data_core.ipynb.
Converted 06_data_source.ipynb.
Converted 07_vision_core.ipynb.
Converted 08_pets_tutorial.ipynb.
Converted 09_vision_augment.ipynb.
Converted 11_layers.ipynb.
Converted 11a_vision_models_xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 14_callback_schedule.ipynb.
Converted 15_callback_hook.ipynb.
Converted 16_callback_progress.ipynb.
Converted 17_callback_tracker.ipynb.
Converted 18_callback_fp16.ipynb.
Converted 19_callback_mixup.ipynb.
Converted 20_metrics.ipynb.
Converted 21_tutorial_imagenette.ipynb.
Converted 22_vision_learner.ipynb.
Converted 23_tutorial_transfer_learning.ipynb.
Converted 30_text_core.ipynb.
Converted 31_text_data.ipynb.
Converted 32_text_models_awdlstm.ipynb.
Converted 33_text_models_core.i